In [1]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import classification_report
import acquire
import prepare
import model_func as m
import pandas as pd
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
from io import StringIO
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn.tree import export_graphviz
import graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

# Exercise

### 1. What is your baseline prediction? What is your baseline accuracy? remember: your baseline prediction for a classification problem is predicting the most prevelant class in the training dataset (the mode). When you make those predictions, what is your accuracy? This is your baseline accuracy.

In [ ]:
#acquire data
df = acquire.get_titanic_data()

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
#split data
train, validate, test = prepare.prep_titanic_data(df, column = 'age', method = 'median', dummies = ['embarked', 'sex'])


In [ ]:
train.shape, validate.shape, test.shape

In [ ]:
train.head(1)

In [ ]:
# drop out non-numerical columns or non-encoded version remaining in this data set.
#in this case, it is only 'passaner_id'
train.drop(columns = ['passenger_id'], inplace=True)
validate.drop(columns = ['passenger_id'], inplace=True)
test.drop(columns = ['passenger_id'], inplace=True)

In [ ]:
train.shape, validate.shape, test.shape

In [ ]:
train.head(2)

In [ ]:
# We will be attempting to make a Decision Tree Classifier Model that will predict survival on the 
# Titanic that performs better than the baseline

In [ ]:
# we are working with survided. The most prevelant class in the training dataset (the mode)
train.survived.value_counts()

In [ ]:
train.survived.mode()

In [ ]:
#so the baselined is survived = 0
train['baseline'] = 0

In [ ]:
#accuracy
(train.baseline == train.survived).mean()

In [ ]:
print(f'The baseline accuracy for nonsurvival in all cases on the Titanic Dataset is {(train.baseline == train.survived).mean():.3}')

In [ ]:
# classification report of baseline model
baseline_class_report = classification_report(train.survived, train.baseline, zero_division=True)
print(baseline_class_report)

In [ ]:
# #other way to check the accuracy of baseline
# #split 
# X_train, y_train = train.drop(columns='survived'), train.survived
# #as the baseline mode is = 0 , thats why we put constant = 0
# # 1. Create the object
# baseline = DummyClassifier(strategy='constant', constant=0)
# # 2. Fit the object
# baseline.fit(X_train, y_train)

In [ ]:
# print(f'Baseline accuracy: %.4f' % baseline.score(X_train, y_train))

### 2. Fit the decision tree classifier to your training sample and transform (i.e. make predictions on the training sample)

In [ ]:
train.head(1)

In [ ]:
# remove baseline assumption from the train, we wont need this column anymore
train.drop(columns='baseline', inplace=True)

In [ ]:
train.head()

In [ ]:
# use the funnction that I create:

# X_train, y_train = model.split_Xy(df = train, column = 'survived')
# X_validate, y_validate = model.split_Xy(validate, 'survived') 
# X_test, y_test = model.split_Xy(test, 'survived') 


In [ ]:
# split our X and y
# do the capital X, lowercase y thing for train test and split
# X is the data frame of the features, y is a series of the target
X_train = train.drop(columns='survived')
y_train = train[['survived']]
X_validate, y_validate = validate.drop(columns='survived'), validate['survived']
X_test, y_test = test.drop(columns='survived'), test['survived']

In [ ]:
# #other way to do the split is decide wich columns we are going to use
# X_cols = [ 'pclass','age', 'sex_male','alone', 'embarked_Q', 'embarked_S']
# y_col = 'survived'

# X_train, y_train = train[X_cols], train[y_col]
# X_train, y_train = train[X_cols], train[y_col]
# X_validate, y_validate = validate[X_cols], validate[y_col]
# X_test, y_test = test[X_cols], test[y_col]

In [ ]:
X_train.head(1)

In [ ]:
y_train.head(1)

In [ ]:
X_train.shape, y_train.shape

In [ ]:
# create the model
model = DecisionTreeClassifier()
# fit the model
model.fit(X_train, y_train)

In [ ]:
#calculate the accuracy
acc = model.score(X_train, y_train)
acc

In [ ]:

# look at the model scores for training set and validate set
print(f'training score: {model.score(X_train, y_train):.2%}')
print(f'validate score: {model.score(X_validate, y_validate):.2%}')

In [ ]:

import matplotlib.pyplot as plt
plt.figure(figsize=(24, 12))
plot_tree(
    model,
    feature_names=X_train.columns.tolist(),
    class_names=['died', 'survived'],
)
plt.show()

In [ ]:
#as the difference between train and validate is big. I will use  max_features and max_depth
#when score.train > score.validate means overfit

In [ ]:
# create the model
model1 = DecisionTreeClassifier(max_depth=3, max_features=3)
# fit the model
model1.fit(X_train, y_train)

In [ ]:
# look at the model scores for training set and validate set
print(f'training score: {model1.score(X_train, y_train):.2%}')
print(f'validate score: {model1.score(X_validate, y_validate):.2%}')

In [ ]:

import matplotlib.pyplot as plt
plt.figure(figsize=(24, 12))
plot_tree(
    model1,
    feature_names=X_train.columns.tolist(),
    class_names=['died', 'survived'],
)
plt.show()

In [ ]:
#prediction based on our model1
prediction = model1.predict(X_train)

In [ ]:
prediction

In [ ]:
pd.Series(prediction).value_counts()

In [ ]:
#confusion matrix
conf_m = confusion_matrix(y_train, prediction )
conf_m

In [ ]:
#check if rows or columns have the actual number. in this case rows has the actual number
train.survived.value_counts()

In [ ]:
 #just to see
mat =  pd.DataFrame ((confusion_matrix(y_train, prediction )),index = ['actual_dead','actual_survived'], columns =['pred_dead','pred_survived' ])
mat

In [ ]:
rubric_df = pd.DataFrame([['True Negative', 'False positive'], ['False Negative', 'True Positive']], columns=mat.columns, index=mat.index)
rubric_df

In [ ]:
r = rubric_df + ': ' + mat.values.astype(str)
r

In [ ]:
type(r)

In [ ]:
conf_m

In [ ]:
conf_m[0,0]

positive = survived

these calculate the rates
- tpr = tp/(tp+fn)
- fpr = fp/(fp+tn)
- tnr = tn/(tn+fp)
- fnr = fn/(fn+tp)

https://en.wikipedia.org/wiki/Confusion_matrix

In [ ]:
#assign the values
tp = conf_m[1,1]
fp =conf_m[0,1] 
fn= conf_m[1,0]
tn =conf_m[0,0]

In [ ]:
#calculate the rate
tpr = tp/(tp+fn)
fpr = fp/(fp+tn)
tnr = tn/(tn+fp)
fnr = fn/(fn+tp)

In [ ]:
#     matrix = confusion_matrix(y, model.predict(X))
#     tpr = matrix[1,1] / (matrix[1,1] + matrix[1,0])
#     fpr = matrix[0,1] / (matrix[0,1] + matrix[0,0])
#     tnr = matrix[0,0] / (matrix[0,0] + matrix[0,1])
#     fnr = matrix[1,0] / (matrix[1,1] + matrix[1,0])

In [ ]:
#calculate the accuracy
acc = model1.score(X_train, y_train)
acc

In [ ]:
print(f'''
The accuracy for our model is {acc:.4}
The True Positive Rate is {tpr:.3}, The False Positive Rate is {fpr:.3},
The True Negative Rate is {tnr:.3}, and the False Negative Rate is {fnr:.3}
''')

In [ ]:
#classification report
clas_rep =pd.DataFrame(classification_report(y_train, prediction, output_dict=True)).T
clas_rep

In [ ]:
pd.Series(dict(zip(X_train.columns, model1.feature_importances_)))

**********************

## other models

In [ ]:
model11 = DecisionTreeClassifier(max_depth=3, max_features= 4)
model11.fit(X_train, y_train)
print(f'training score: {model11.score(X_train, y_train):.2%}')


In [ ]:

import matplotlib.pyplot as plt
plt.figure(figsize=(24, 12))
plot_tree(
    model11,
    feature_names=X_train.columns.tolist(),
    class_names=['died', 'survived'],
)
plt.show()

In [ ]:
pd.Series(dict(zip(X_train.columns, model11.feature_importances_)))

In [ ]:

model2 = DecisionTreeClassifier(max_depth=3)
model2.fit(X_train, y_train)

print(f'training score: {model2.score(X_train, y_train):.2%}')
print(f'validate score: {model2.score(X_validate, y_validate):.2%}')

In [ ]:
# we want traning score < validate score.
# an aceptable range is a diff of 5%

In [ ]:

import matplotlib.pyplot as plt
plt.figure(figsize=(24, 12))
plot_tree(
    model2,
    feature_names=X_train.columns.tolist(),
    class_names=['died', 'survived'],
)
plt.show()

In [ ]:
pd.Series(dict(zip(X_train.columns, model2.feature_importances_)))

### 3. Evaluate your in-sample results using the model score, confusion matrix, and classification report.

In [ ]:
# ****************compute metrics on the TRAIN set

In [ ]:
#prediction based on our model
prediction = model.predict(X_train)

In [ ]:
prediction

In [ ]:
# check out the values in the predictions. we cannot use value_counts in array thats whay we conver it to pd.series
pd.Series(prediction).value_counts()

In [ ]:
#model Score: Accuracy

acc = model1.score(X_train, y_train)
acc

In [ ]:
train.head(1)

In [ ]:
#these are the actual numbers
train.survived.value_counts()

In [ ]:
#check if you change the order of labels

In [ ]:
confusion_matrix(y_train, prediction)

In [ ]:
confusion_matrix(y_train, prediction, labels=(1, 0) ) 

In [ ]:
confusion_matrix(y_train, prediction, labels=(0, 1) )#this is the default

In [ ]:
#the rows are the actual dead  because 0 + 307 = 307, 
confusion_matrix(y_train, prediction)

In [ ]:
cm= pd.DataFrame ((confusion_matrix(y_train, prediction )),index = ['actual_dead','actual_survived'], columns =['pred_dead','pred_survived' ])

In [ ]:
cm

In [ ]:
#model Score: Accuracy

acc = model1.score(X_train, y_train)
acc

In [ ]:
pd.DataFrame(classification_report(y_train, prediction, output_dict=True)).T

### 4. Compute: Accuracy, true positive rate, false positive rate, true negative rate, false negative rate, precision, recall, f1-score, and support.

In [ ]:
#### on the validate set *****


- positive is : survived
- True Positive: predict survived and it is survived
- True Negative: predict death and it is death
- False Positive: predict survived and it is death
- False Negative: predict death and it is survived

In [ ]:
# print the model score on the validate set
print(f'validate score: {model1.score(X_validate, y_validate):.2%}')

In [ ]:
val_pred = model1.predict(X_validate)

In [ ]:
validate.survived.value_counts()

In [ ]:
confusion_matrix(validate.survived, val_pred)

In [ ]:
cm2= confusion_matrix(y_validate, val_pred)

In [ ]:
#just to see
pd.DataFrame (confusion_matrix(y_validate, val_pred),index = ['actual_death','actual_survived'], columns =['pred_death','pred_survived' ] )

In [ ]:
#psitive : survived
#assign the values
tp = cm2[1,1]
fp =cm2[0,1] 
fn= cm2[1,0]
tn =cm2[0,0]

In [ ]:
#calculate the rate
tpr = tp/(tp+fn)
fpr = fp/(fp+tn)
tnr = tn/(tn+fp)
fnr = fn/(fn+tp)

In [ ]:
print(f'''

The True Positive Rate is {tpr:.3}, The False Positive Rate is {fpr:.3},
The True Negative Rate is {tnr:.3}, and the False Negative Rate is {fnr:.3}
''')

In [ ]:
#accuracy
acc_model1 =accuracy_score(y_validate, val_pred)
acc_model1

In [ ]:
#precision
prec_model1= precision_score(y_validate, val_pred, pos_label = 1 )
prec_model1

In [ ]:
#recall
rec_model1= recall_score(y_validate, val_pred, pos_label = 1 )
rec_model1

In [ ]:
df = pd.DataFrame(classification_report(y_validate, val_pred, output_dict= True))
df

In [ ]:
df.rename(columns={'0': "dead", '1': "survived"}, inplace = True)

In [ ]:
df.T

### 5. Run through steps 2-4 using a different max_depth value.

In [ ]:

model2 = DecisionTreeClassifier(max_depth=2)
model2.fit(X_train, y_train)

print(f'training score: {model2.score(X_train, y_train):.2%}')
print(f'validate score: {model2.score(X_validate, y_validate):.2%}')

In [ ]:
pd.Series(dict(zip(X_train.columns, model2.feature_importances_)))

In [ ]:
#using my function import model 
m.model_performs(X_train, y_train, model2)

In [ ]:
#if I want to see a dec_tree
m.dec_tree(model2, X_train)

In [ ]:
model3 = DecisionTreeClassifier(max_depth= 5)
model3.fit(X_train, y_train)

print(f'training score: {model3.score(X_train, y_train):.2%}')
print(f'validate score: {model3.score(X_validate, y_validate):.2%}')

In [ ]:
#using my function import model 
m.model_performs(X_train, y_train, model3)

In [ ]:
#if I want to see a dec_tree
m.dec_tree(model3, X_train)

In [ ]:
#accuracy
acc_model2 =accuracy_score(train.survived, train.prediction_2)

In [ ]:
#precision
prec_model2 =precision_score(train.survived, train.prediction_2, pos_label = 1 )

In [ ]:
#recall
rec_model2= recall_score(train.survived, train.prediction_2, pos_label = 1 )

### 6. Which model performs better on your in-sample data? (aka TRAIN set)

In [ ]:
# model1 using my func
model1 = DecisionTreeClassifier(max_depth=3, max_features=3)

model1.fit(X_train, y_train)

In [ ]:
m.model_performs(X_train, y_train, model1)

In [ ]:
#if I want to see a dec_tree
model.dec_tree(model1, X_train)

### 7.Which model performs best on your out-of-sample data, the validate set? (aka the VALIDATE set)

In [ ]:
# get predictions for our validation sets
val_pred_1 = model1.predict(X_validate)
val_pred_2 = model2.predict(X_validate)
val_pred_3 = model3.predict(X_validate)


In [ ]:
# get validation accuracy
acc_v_1 = model1.score(X_validate, y_validate)
acc_v_2 = model2.score(X_validate, y_validate)
acc_v_3 = model3.score(X_validate, y_validate)


In [ ]:
acc_v_1, acc_v_2, acc_v_3

In [ ]:
# look at the model scores for training set and validate set
print(f'training score: {model1.score(X_train, y_train):.2%}')
print(f'validate score: {model1.score(X_validate, y_validate):.2%}')

In [ ]:

# look at the model scores for training set and validate set
print(f'training score: {model1.score(X_train, y_train):.2%}')
print(f'validate score: {model1.score(X_validate, y_validate):.2%}')

In [ ]:

# look at the model scores for training set and validate set
print(f'training score: {model2.score(X_train, y_train):.2%}')
print(f'validate score: {model2.score(X_validate, y_validate):.2%}')

In [ ]:
# look at the model scores for training set and validate set
print(f'training score: {model3.score(X_train, y_train):.2%}')
print(f'validate score: {model3.score(X_validate, y_validate):.2%}')

In [ ]:
:
# 
# 
# We see here a very significant drop-off on model ,
# where we did not specify a maximum depth, 
# suggesting it was very overfit to the training set
# training score >>> validate score = overfit
# b
# 
#

In [ ]:
dot_data = export_graphviz(model1, feature_names= X_train.columns, rounded=True, filled=True, out_file=None)
graph = graphviz.Source(dot_data)

In [ ]:
graph.render('titanic_model_1_tree', view=True)

In [ ]:
dot_data = export_graphviz(model3, feature_names= X_train.columns, rounded=True, filled=True, out_file=None)
graph = graphviz.Source(dot_data)
graph.render('titanic_model_1_tree', view=True)

# Random Forest

### 1. Fit the Random Forest classifier to your training sample and transform (i.e. make predictions on the training sample) setting the random_state accordingly and setting min_samples_leaf = 1 and max_depth = 10.

In [ ]:
#acquire data
df = acquire.get_titanic_data()

In [ ]:
#split data
train, validate, test = prepare.prep_titanic_data(df, column = 'age', method = 'median', dummies = ['embarked', 'sex'])


In [ ]:
# drop out non-numerical columns or non-encoded version remaining in this data set.
#in this case, it is only 'passaner_id'
train.drop(columns = ['passenger_id'], inplace=True)
validate.drop(columns = ['passenger_id'], inplace=True)
test.drop(columns = ['passenger_id'], inplace=True)

In [ ]:
# use the funnction that I create:

X_train, y_train = m.split_Xy(df = train, column = 'survived')
X_validate, y_validate = m.split_Xy(validate, 'survived') 
X_test, y_test = m.split_Xy(test, 'survived') 

In [ ]:
# Establish our baseline.  The rate at which the assumption of the majority class matches the real values.  
#If a model does not perform better than this, it would not be wise to deploy.
baseline = (y_train.value_counts().idxmax() == y_train).mean()
baseline

In [ ]:
# create the Random Forest Model
rfmodel = RandomForestClassifier(min_samples_leaf=1, max_depth=10, random_state=1349)

In [ ]:
# fit the thing
rfmodel.fit(X_train, y_train)

In [ ]:
#this is the next step but I already have it in my function
#  use the thing
# y_pred = rfmodel.predict(X_train)

### 2. Evaluate your results using the model score, confusion matrix, and classification report.

In [ ]:
m.model_performs(X_train, y_train, rfmodel)

### 3. Print and clearly label the following: Accuracy, true positive rate, false positive rate, true negative rate, false negative rate, precision, recall, f1-score, and support.

In [ ]:
m.model_performs(X_train, y_train, rfmodel)

### 4. Run through steps increasing your min_samples_leaf and decreasing your max_depth.

In [ ]:
# create the Random Forest Model
rfmod2 = RandomForestClassifier(min_samples_leaf=4, max_depth=4, random_state=1349)

# fit the model
rfmod2.fit(X_train, y_train)

In [ ]:
m.model_performs(X_train, y_train, rfmod2)

### 5. What are the differences in the evaluation metrics? Which performs better on your in-sample data? Why?

In [ ]:
m.model_performs(X_validate, y_validate, rfmodel)

model.model_performs(X_validate, y_validate, rfmod2)

### After making a few models, which one has the best performance (or closest metrics) on both train and validate?

# KNN Exercises

### 1. Fit a K-Nearest Neighbors classifier to your training sample and transform (i.e. make predictions on the training sample)

In [ ]:
X_train.head(1)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, y_train)

In [ ]:
# look at the model scores for training set and validate set
print(f'training score: {knn.score(X_train, y_train):.2%}')
print(f'validate score: {knn.score(X_validate, y_validate):.2%}')

In [ ]:
for k in range(1, 15):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    print(f'Model: {k} when k is {k}')
    print(f'training score: {knn.score(X_train, y_train):.2%}')
    print(f'validate score: {knn.score(X_validate, y_validate):.2%}')
    print('__________________________________________________________')


In [ ]:
#I'm going to 
knn2 = KNeighborsClassifier(14)
knn2.fit(X_train, y_train)

In [ ]:
# look at the model scores for training set and validate set
print(f'training score: {knn2.score(X_train, y_train):.2%}')
print(f'validate score: {knn2.score(X_validate, y_validate):.2%}')

m.model_performs(X_train, y_train, knn)

### 2. Evaluate your results using the model score, confusion matrix, and classification report.

In [ ]:
#for knn model
m.model_performs(X_train, y_train, knn)

In [ ]:
#for knn2 model
m.model_performs(X_train, y_train, knn2)

### 3. Print and clearly label the following: Accuracy, true positive rate, false positive rate, true negative rate, false negative rate, precision, recall, f1-score, and support.

In [ ]:
m.model_performs(X_train, y_train, knn2)

### 4. Run through steps 2-4 setting k to 10

In [ ]:
knn3 = KNeighborsClassifier(10)
knn3.fit(X_train, y_train)

In [ ]:
m.model_performs(X_train, y_train, knn3)

### 5. Run through setps 2-4 setting k to 20

In [ ]:
knn4 = KNeighborsClassifier(20)
knn4.fit(X_train, y_train)

In [ ]:
m.model_performs(X_train, y_train, knn4)

### 6. What are the differences in the evaluation metrics? Which performs better on your in-sample data? Why?

In [ ]:
model 2

### 7. Which model performs best on our out-of-sample data from validate?

In [ ]:
#model 2
print(f'training score: {knn2.score(X_train, y_train):.2%}')
print(f'validate score: {knn2.score(X_validate, y_validate):.2%}')

In [ ]:
m.model_performs(X_validate, y_validate, knn2)

In [ ]:
m.compare (knn, knn2, X_train,y_train)

#  Logistic Regression

### 1. Create a model that includes age in addition to fare and pclass. Does this model perform better than your baseline?

In [2]:
#acquire data
df = acquire.get_titanic_data()

In [3]:
#split data
train, validate, test = prepare.prep_titanic_data(df, column = 'age', method = 'median', dummies = ['embarked', 'sex'])


In [4]:
train.head()

,passenger_id,survived,pclass,age,sibsp,parch,fare,alone,embarked_Q,embarked_S,sex_male
583,583,0,1,36.0,0,0,40.1250,1,0,0,1
165,165,1,3,9.0,0,2,20.5250,0,0,1,1
50,50,0,3,7.0,4,1,39.6875,0,0,1,1
259,259,1,2,50.0,0,1,26.0000,0,0,1,0
306,306,1,1,28.0,0,0,110.8833,1,0,0,0


In [5]:
train.shape

(498, 11)

In [6]:
#other way to do the split is decide wich columns we are going to use
X_cols = [ 'age','pclass', 'fare']
y_col = 'survived'

X_train, y_train = train[X_cols], train[y_col]
X_validate, y_validate = validate[X_cols], validate[y_col]
X_test, y_test = test[X_cols], test[y_col]

In [7]:
X_train.shape, X_validate.shape, X_test.shape

((498, 3), (214, 3), (179, 3))

In [8]:
#so the baselined is survived = 0
train['baseline'] = 0

In [9]:
print(f'The baseline accuracy for nonsurvival in all cases on the Titanic Dataset is {(train.baseline == train.survived).mean():.3}')

The baseline accuracy for nonsurvival in all cases on the Titanic Dataset is 0.616


In [10]:
logit =LogisticRegression(C=1.0 , random_state=123)
logit.fit(X_train, y_train)

LogisticRegression(random_state=123)

In [11]:
# now use the model to make predictions
y_pred = logit.predict(X_train)

In [12]:
print('Accuracy: {:.2f}'.format(logit.score(X_train, y_train)))

Accuracy: 0.70


In [13]:
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           0       0.71      0.87      0.78       307
           1       0.67      0.43      0.53       191

    accuracy                           0.70       498
   macro avg       0.69      0.65      0.66       498
weighted avg       0.70      0.70      0.69       498



In [14]:

print(f'training score: {logit.score(X_train, y_train):.2%}')
print(f'validate score: {logit.score(X_validate, y_validate):.2%}')

training score: 70.28%
validate score: 71.03%


### 2. Include sex in your model as well. Note that you'll need to encode or create a dummy variable of this feature before including it in a model.

In [15]:
X_cols = [ 'age','pclass', 'fare', 'sex_male']
y_col = 'survived'

X_train, y_train = train[X_cols], train[y_col]
X_validate, y_validate = validate[X_cols], validate[y_col]
X_test, y_test = test[X_cols], test[y_col]

In [16]:

logit2 =LogisticRegression(C=1.0 , random_state=123)
logit2.fit(X_train, y_train)

LogisticRegression(random_state=123)

In [17]:
# now use the model to make predictions
y_pred = logit2.predict(X_train)

In [18]:

print(f'training score: {logit2.score(X_train, y_train):.2%}')
print(f'validate score: {logit2.score(X_validate, y_validate):.2%}')

training score: 80.72%
validate score: 78.04%


In [19]:
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.85      0.85       307
           1       0.76      0.73      0.74       191

    accuracy                           0.81       498
   macro avg       0.80      0.79      0.79       498
weighted avg       0.81      0.81      0.81       498



### 3. Try out other combinations of features and models.

In [20]:
logit3 =LogisticRegression(C=1 ,class_weight='balanced', random_state=123)
logit3.fit(X_train, y_train)

LogisticRegression(C=1, class_weight='balanced', random_state=123)

In [21]:
# now use the model to make predictions
y_pred = logit3.predict(X_train)

In [22]:
print(f'training score: {logit3.score(X_train, y_train):.2%}')
print(f'validate score: {logit3.score(X_validate, y_validate):.2%}')

training score: 80.12%
validate score: 75.70%


In [23]:
logit4 =LogisticRegression(penalty='l2',C=0.5, random_state=123, class_weight={0:61, 1:39})
logit4.fit(X_train, y_train)

LogisticRegression(C=0.5, class_weight={0: 61, 1: 39}, random_state=123)

In [24]:
print(f'training score: {logit4.score(X_train, y_train):.2%}')
print(f'validate score: {logit4.score(X_validate, y_validate):.2%}')

training score: 79.72%
validate score: 76.64%


### 4. Use you best 3 models to predict and evaluate on your validate sample.

In [25]:
logit2 =LogisticRegression(C=1.0 , random_state=123)
logit2.fit(X_train, y_train)

LogisticRegression(random_state=123)

In [26]:
logit4 =LogisticRegression(penalty='l2',C=0.5, random_state=123, class_weight={0:61, 1:39})
logit4.fit(X_train, y_train)

LogisticRegression(C=0.5, class_weight={0: 61, 1: 39}, random_state=123)

In [27]:
m.compare(logit2, logit4, X_validate, y_validate)


    ******       Model 1  ******                                ******     Model 2  ****** 
    The accuracy for our model 1 is 78.0374%            |   The accuracy for our model 2 is 76.6355%  
                                                        |
    The True Positive Rate is 68.293%                   |   The True Positive Rate is 56.098%  
    The False Positive Rate is 15.909%                  |   The False Positive Rate is 10.606% 
    The True Negative Rate is 84.091%                   |   The True Negative Rate is 89.394% 
    The False Negative Rate is 31.707%                  |   The False Negative Rate is 43.902%

    _____________________________________________________________________________________________________________
    

    The positive is  'survived'

    Confusion Matrix
    


,pred_dead,pred_survived
actual_dead,True Negative: 111,False positive: 21
actual_survived,False Negative: 26,True Positive: 56
,pred_dead,pred_survived
actual_dead,True Negative: 118,False positive: 14
actual_survived,False Negative: 36,True Positive: 46




    ________________________________________________________________________________
    
    Classification Report:
    


,precision,recall,f1-score,support
dead,0.810219,0.840909,0.825279,132.000000
survived,0.727273,0.682927,0.704403,82.000000
accuracy,0.780374,0.780374,0.780374,0.780374
macro avg,0.768746,0.761918,0.764841,214.000000
weighted avg,0.778436,0.780374,0.778962,214.000000
,precision,recall,f1-score,support
dead,0.766234,0.893939,0.825175,132.000000
survived,0.766667,0.560976,0.647887,82.000000
accuracy,0.766355,0.766355,0.766355,0.766355
macro avg,0.766450,0.727458,0.736531,214.000000


### 5. Choose you best model from the validation performation, and evaluate it on the test dataset. How do the performance metrics compare to validate? to train?

In [ ]:
# I select logit2

In [28]:
#compare Train and Test
m.compare_metrics(logit2,'Train',X_train, y_train,'Test', X_test, y_test)


    ******    Train       ******                              ******     Test    ****** 
       Overall Accuracy:  80.72%              |                Overall Accuracy:  79.89%  
                                                
    True Positive Rate:  73.30%               |          The True Positive Rate:  73.91%  
    False Positive Rate:  14.66%              |          The False Positive Rate:  16.36% 
    True Negative Rate:  85.34%               |          The True Negative Rate:  83.64% 
    False Negative Rate:  26.70%              |          The False Negative Rate:  26.09%
    _________________________________________________________________________________
    

    Positive =  'survived'
    Confusion Matrix
    


,pred_dead,pred_survived
actual_dead,TN : 262,FP : 45
actual_survived,FN : 51,TP : 140
,pred_dead,pred_survived
actual_dead,TN : 92,FP : 18
actual_survived,FN : 18,TP : 51



    ________________________________________________________________________________
    
    Classification Report:
    


,precision,recall,f1-score,support
dead,0.837061,0.853420,0.845161,307.000000
survived,0.756757,0.732984,0.744681,191.000000
accuracy,0.807229,0.807229,0.807229,0.807229
macro avg,0.796909,0.793202,0.794921,498.000000
weighted avg,0.806261,0.807229,0.806624,498.000000
,precision,recall,f1-score,support
dead,0.836364,0.836364,0.836364,110.000000
survived,0.739130,0.739130,0.739130,69.000000
accuracy,0.798883,0.798883,0.798883,0.798883
macro avg,0.787747,0.787747,0.787747,179.000000


In [29]:
#compare validate and Test
m.compare_metrics(logit2,'Validate',X_validate, y_validate,'Test', X_test, y_test)



    ******    Validate       ******                              ******     Test    ****** 
       Overall Accuracy:  78.04%              |                Overall Accuracy:  79.89%  
                                                
    True Positive Rate:  68.29%               |          The True Positive Rate:  73.91%  
    False Positive Rate:  15.91%              |          The False Positive Rate:  16.36% 
    True Negative Rate:  84.09%               |          The True Negative Rate:  83.64% 
    False Negative Rate:  31.71%              |          The False Negative Rate:  26.09%
    _________________________________________________________________________________
    

    Positive =  'survived'
    Confusion Matrix
    


,pred_dead,pred_survived
actual_dead,TN : 111,FP : 21
actual_survived,FN : 26,TP : 56
,pred_dead,pred_survived
actual_dead,TN : 92,FP : 18
actual_survived,FN : 18,TP : 51



    ________________________________________________________________________________
    
    Classification Report:
    


,precision,recall,f1-score,support
dead,0.810219,0.840909,0.825279,132.000000
survived,0.727273,0.682927,0.704403,82.000000
accuracy,0.780374,0.780374,0.780374,0.780374
macro avg,0.768746,0.761918,0.764841,214.000000
weighted avg,0.778436,0.780374,0.778962,214.000000
,precision,recall,f1-score,support
dead,0.836364,0.836364,0.836364,110.000000
survived,0.739130,0.739130,0.739130,69.000000
accuracy,0.798883,0.798883,0.798883,0.798883
macro avg,0.787747,0.787747,0.787747,179.000000


In [30]:
#compare validate and Test
m.compare_metrics(logit2,'Validaccte',X_validate, y_validate,'Test', X_test, y_test)

incorrect name
